# ![title](Figures/cat2.png)
# PURR Processing Utility for Rapid Results

Import all necessary libraries

In [1]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets
from IPython.display import display

import warnings
from IPython.display import display, Image, clear_output
import time

##Custom SCripts

from MRM_parser import full_parse

from MRM_parser import prep_edge_R

from MRM_parser import make_pie_chart

from MRM_parser import average_pie_chart

from MRM_parser import make_bar_plot_comparisons

from MRM_parser import process_for_biopan

import os
from IPython.display import display
from ipywidgets import widgets


Pre_folder = './Projects/'


from tools.GUI import assign_blank, load_blank_name
from tools.GUI import folder_navigator, load_project_folder

from tools.GUI import filter_samples, load_data_labels
from tools.GUI import display_pair_widgets, load_data, remove_empty_entries, get_unique_json_objects


from tools.parsing import add_suffix

In [4]:
##Add a way to include a custom color scheme




file_name_to_save = 'test'

extra_name = ""

tolerance = 0.3
remove_std = True
save_data= True


load_previously_parsed = False
custom_data=False


In [22]:

folder_navigator()

Project_Folder = load_project_folder()
folder_name_to_save = Project_Folder+ 'Processed Results/'
data_base_name_location = 'lipid_database/Lipid_Database.xlsx'
mzml_folder = Project_Folder +"mzml/"
Pre_edge_r_path = Project_Folder+"Pre_EdgeR/"
plots_2_save_path = Project_Folder+"Plots/"
label_file = Project_Folder+"Labels/labels.csv"
labels_df = pd.read_csv(label_file)

##determines blank
unique_samples = labels_df['Sample Name'].unique()
assign_blank(unique_samples)
blank_name = load_blank_name()

# Assuming labels_df is your DataFrame
filter_samples(labels_df)
labels_df2 = load_data_labels()
labels_df2 = labels_df2.drop(["Sample Name","Position"],axis=1)
labels_list = list(labels_df)
labels_list = labels_list +["Class","Lipid"]





Button(description='Navigate', style=ButtonStyle())

Button(description='Select this folder', style=ButtonStyle())

Button(description='Select Current Folder', style=ButtonStyle())

Select(options=('/home/iyer95/CLAW/CLAW/lipid_platform/Projects',), rows=10, value='/home/iyer95/CLAW/CLAW/lip…

Output()

...

Button(description='Assign Blank', style=ButtonStyle())

Output()

SelectMultiple(description='Samples', index=(0,), options=('11162023_Blank-AMP_2x_MRMs_O3off_01', '11162023_WT…

Button(description='Filter Samples', style=ButtonStyle())

Output()

In [23]:
main_json = {col: labels_df2[col].unique().tolist() for col in labels_df2}

display_pair_widgets(main_json)

In [27]:

json_list_pairs = load_data()
json_list_pairs = remove_empty_entries(json_list_pairs)
json_list_singles = get_unique_json_objects(json_list_pairs)
# make_pie_chart_no_replicates(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# average_pie_chart_no_repeats(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# # make_bar_plot_comparisons(df_matched, plots_2_save_path, json_list_pairs,labels_list,blank_name,extra_name)
# labels_list = labels_list + ['method_type',"Transition"]
# df_matched = add_suffix(df_matched)
# combined_df = prep_edge_R(df_matched,json_list_pairs,Pre_edge_r_path,blank_name,labels_list,extra_name)
print(json_list_pairs)
print(json_list_singles)

[[{'Biology': ['cereb'], 'Genotype': ['5xFAD']}, {'Biology': ['cereb'], 'Genotype': ['WT']}], [{'Biology': ['cortex'], 'Genotype': ['5xFAD']}, {'Biology': ['cortex'], 'Genotype': ['WT']}], [{'Biology': ['dienc'], 'Genotype': ['5xFAD']}, {'Biology': ['dienc'], 'Genotype': ['WT']}], [{'Biology': ['hippo'], 'Genotype': ['5xFAD']}, {'Biology': ['hippo'], 'Genotype': ['WT']}], [{'Genotype': ['5xFAD']}, {'Genotype': ['WT']}], [{}, {}]]
[{'Genotype': ['5xFAD']}, {'Genotype': ['WT']}, {'Biology': ['cortex'], 'Genotype': ['5xFAD']}, {'Biology': ['cereb'], 'Genotype': ['5xFAD']}, {'Biology': ['cortex'], 'Genotype': ['WT']}, {'Biology': ['hippo'], 'Genotype': ['WT']}, {'Biology': ['hippo'], 'Genotype': ['5xFAD']}, {'Biology': ['dienc'], 'Genotype': ['WT']}, {'Biology': ['cereb'], 'Genotype': ['WT']}, {}, {'Biology': ['dienc'], 'Genotype': ['5xFAD']}]


In [25]:
# Ignore warnings
warnings.filterwarnings('ignore')

# # Display GIF
# gif = Image(filename='Figures/cat_gif.gif') # replace 'your_gif.gif' with the path to your GIF
# display(gif)

print("Your data is PURRing...")

if load_previously_parsed == True:
    df_matched = pd.read_csv(Project_Folder+"Processed Results/"+file_name_to_save+".csv")
    

    
#     tag_values = ['TAG14', 'TAG16', 'TAG18', 'TAG20', 'TAG22']
#     dag_values = ['DAG16', 'DAG18']

#     df_matched.loc[df_matched['method_type'].isin(tag_values), 'Class'] = 'TAG'
#     df_matched.loc[df_matched['method_type'].isin(dag_values), 'Class'] = 'DAG'
    
else:
    df_matched = full_parse(data_base_name_location,mzml_folder, folder_name_to_save,labels_df, blank_name, 
                            file_name_to_save,tolerance, custom_data = custom_data, remove_std = remove_std,save_data=save_data)
#     tag_values = ['TAG14', 'TAG16', 'TAG18', 'TAG20', 'TAG22']
#     dag_values = ['DAG16', 'DAG18']

#     df_matched.loc[df_matched['method_type'].isin(tag_values), 'Class'] = 'TAG'
#     df_matched.loc[df_matched['method_type'].isin(dag_values), 'Class'] = 'DAG'
    
print("Complete")






json_list_pairs = load_data()
json_list_pairs = remove_empty_entries(json_list_pairs)
json_list_singles = get_unique_json_objects(json_list_pairs)
# make_pie_chart_no_replicates(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# average_pie_chart_no_repeats(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# make_bar_plot_comparisons(df_matched, plots_2_save_path, json_list_pairs,labels_list,blank_name,extra_name)
labels_list = labels_list + ['method_type',"Transition"]
df_matched = add_suffix(df_matched)
combined_df = prep_edge_R(df_matched,json_list_pairs,Pre_edge_r_path,blank_name,labels_list,extra_name)
# process_for_biopan(Project_Folder)
# !bash myjob.sh


Your data is PURRing...


FileNotFoundError: [Errno 2] No such file or directory: '/home/iyer95/CLAW/CLAW/lipid_platform/Projects/AMP/mzml/'